# 一. 分析区域简介

我在本次数据清洗项目中选择的区域是中国贵州省贵阳市，是中国西南地区新兴的国家级大数据试验区。之所以选择该地区，一方面是因为它是我的家乡，我比较熟悉；另一方面课程中的案例研究使用的都是外国的城市，所以我选择一个中国的城市进行数据清洗和导入工作，想看看在不同文化环境下有什么样的不同。

地图位置及数据导出：

* [地图位置](https://www.openstreetmap.org/relation/2782246#map=9/26.7750/106.6965)
* [数据导出](http://overpass-api.de/api/map?bbox=105.1996,25.8469,108.1934,27.6981)

# 二. 数据整理

整个OSM文件有66M，总共840383行。我先使用Udacity项目提供的代码（详见samplek.py）从原始数据中分别抽取小元素样本和大元素样本，然后运行data.py将样本数据转为csv文件并对数据进行人工观察。发现地图数据中存在以下问题。

1. 一些节点的经纬度不在有效边界范围内（多个数据项之间不匹配，不满足数据质量评估中的“一致性”原则）；
2. 一部分道路的英文名称出现了多种英文缩写（数据模式不统一，不满足数据质量评估中的“有效性”原则）；
3. 一些节点同时具有zh和name两个属性，但是名称不一致（多个数据项之间不匹配，不满足数据质量评估中的“一致性”原则）；
4. 一些节点的name属性是纯中文，但另一些又是中文+英文（数据模式不统一，不满足数据质量评估中的“有效性”原则）。

## 1. 节点经纬度问题

In [2]:
import xml.etree.cElementTree as ET
import math

osm_file = 'guiyang_china.osm'

# number of nodes whose latitude is out of bounds
count_of_error_lat = 0
# number of nodes whose longitude is out of bounds
count_of_error_lon = 0
# the maximum difference of latitude
max_abs_lat_diff = -1
# the maximum difference of longitude
max_abs_lon_diff = -1
# the minimum difference of latitude
min_abs_lat_diff = float('inf')
# the minimum difference of longitude
min_abs_lon_diff = float('inf')

for event, element in ET.iterparse(osm_file, events=("start",)):
    # retrieve the latitude and longitude range from <bounds .../>
    if element.tag == 'bounds':
        minlat = float(element.attrib['minlat'])
        minlon = float(element.attrib['minlon'])
        maxlat = float(element.attrib['maxlat'])
        maxlon = float(element.attrib['maxlon'])

    # only node elements have latitude and longitude information
    if element.tag == 'node':
        lat = float(element.attrib['lat'])
        lon = float(element.attrib['lon'])
        if not (minlat <= lat <= maxlat):
            diff = min(math.fabs(lat-minlat), math.fabs(lat-maxlat))
            # print('node {} latitude {} out of bounds [{}, {}], diff {}'.format(element.attrib['id'], lat, minlat, maxlat, diff))
            count_of_error_lat += 1
            if diff > max_abs_lat_diff:
                max_abs_lat_diff = diff
            if diff < min_abs_lat_diff:
                min_abs_lat_diff = diff
        if not (minlon <= lon <= maxlon):
            diff = min(math.fabs(lat - minlat), math.fabs(lat - maxlat))
            # print('node {} longitude {} out of bounds [{}, {}], diff {}'.format(element.attrib['id'], lon, minlon, maxlon, diff))
            count_of_error_lon += 1
            if diff > max_abs_lon_diff:
                max_abs_lon_diff = diff
            if diff < min_abs_lon_diff:
                min_abs_lon_diff = diff

print('total number of error lat {}, lon {}'.format(count_of_error_lat, count_of_error_lon))
print('latitude diff range [{}, {}], longitude diff range [{}, {}]'.format(min_abs_lat_diff, max_abs_lat_diff, min_abs_lon_diff, max_abs_lon_diff))

total number of error lat 8497, lon 2703
latitude diff range [2.00000002337e-07, 4.3298604], longitude diff range [1.20000000017e-05, 4.3298604]


可以看到总体数据中有8497条数据纬度超出边界范围，另有2703条数据经度超出边界范围，数量上看还是很客观的，看来这部分数据存在问题。但进一步分析发现无论是经度还是纬度其误差范围最大都不超过5度，考虑到GPS等仪器产生的定位误差，我认为这部分数据误差对后续的分析工作不造成影响，可以不做处理。

## 2. 道路英文缩写不统一

## 3. name属性格式不统一

## 4. zh属性和name属性不一致

# 三. 数据概况和探索

# 四. 数据改进建议

# 五. 结论

# 代码功能性

## 代码是否发挥期望的功能？
最终项目代码的功能性与项目文件中描述的一致。

# 项目可读性

## 项目是否使用良好的编码实践？
最终项目代码遵从直观、易于跟进的逻辑结构。

## 代码的注释是否有用且无赘言？
最终项目代码直观可读，且标有清晰注释。

# 你的地图中遇到的问题

## 项目是否记录了数据整理中遇到的挑战？
学生的回复表现出对审计过程、纠正或规范化数据方式的理解，包括应对特定于位置的问题，例如与语言或传统格式化方式相关。

## 数据是否通过编程进行了清理？
数据审计中遇到的某些问题通过编程进行了清理。

# 数据概述

## OSM XML 是否足够大？
OSM XML 未压缩时至少有 50 MB。

## 是否计算了数据集的概述统计？
使用了数据库查询来提供数据库的统计概述，如：

* 文件大小
* 唯一用户的数量
* 节点和途径的数量
* 所选节点类型（如：咖啡店、商店等）的数量

此外，还计算了不在上述列表中的统计数据。对于 SQL 提交，某些查询使用超过一个表。

## 是否对数据库查询进行了书面记录？
提交文件包含上述数据库查询及统计数据。

# 关于数据集的其他想法

## 是否包含额外改进建议？
提交文件包含一个或多个用于改进或分析数据的额外建议。

## 是否讨论了额外改进的益处和问题？
提交文件中详尽地讨论了实施改进的益处及一些预期的问题。

# 提交的全面性和简洁性

## 提交是否足够长，能够全面回答提出的问题？
提交文件足够长，能够全面回答所提出的问题，同时未给予不必要的细节。一般的良好准则是问题回复的长度在 3 到 6 页之间。